# Molecular Docking Performance Analysis with OpenMP Parallelization

This notebook implements and analyzes molecular docking simulations using both single-threaded and parallel approaches. We'll compare performance metrics across different thread counts and visualize the results.

## Setup and Dependencies


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import psutil
import os
import subprocess
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import resource

# Configure OpenMP environment
os.environ["OMP_PLACES"] = "cores"
os.environ["OMP_PROC_BIND"] = "spread"

# Create directories for prepared files
os.makedirs("data/Receptors/prepared_receptors", exist_ok=True)
os.makedirs("data/Ligands/prepared_ligands", exist_ok=True)

## Prepare Input Files

In [3]:
# File preparation utilities
def convert_sdf_to_pdbqt(input_file, output_dir):
    """Convert SDF files to PDBQT format using OpenBabel.
    
    This function is specifically designed for converting ligand files from PubChem's SDF format
    to the PDBQT format required by AutoDock Vina. It handles the necessary preprocessing steps
    including adding hydrogens and generating 3D coordinates.
    """
    try:
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        output_file = Path(output_dir) / f"{Path(input_file).stem}.pdbqt"
        
        cmd = f"obabel {input_file} -O {output_file} -h --gen3d --minimize"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"Successfully converted {input_file} to {output_file}")
            return str(output_file)
        else:
            print(f"Error during conversion: {result.stderr}")
            return None
    except Exception as e:
        print(f"Unexpected error during conversion: {e}")
        return None

def prepare_receptor_pdbqt(input_file, output_dir):
    """Convert PDB files to PDBQT format for receptors using OpenBabel.
    
    This function handles the special requirements for protein structures, including
    proper protonation states and handling of protein flexibility.
    """
    try:
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        output_file = Path(output_dir) / f"{Path(input_file).stem}.pdbqt"
        
        cmd = f"obabel {input_file} -O {output_file} -h -p 7.4 -xr"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"Successfully converted {input_file} to {output_file}")
            return str(output_file)
        else:
            print(f"Error during conversion: {result.stderr}")
            return None
    except Exception as e:
        print(f"Unexpected error during conversion: {e}")
        return None

## Molecular Docking Implementation

Class to handle molecular docking simulations with both single-threaded and parallel implementations.


In [4]:
class MolecularDocking:
    def __init__(self, receptor_file, ligand_file, receptor_type="BACE1"):
        """Initialize the molecular docking simulation parameters.
        
        This enhanced initialization supports Alzheimer's-specific receptors and includes
        automatic file preparation if needed.

        Args:
            receptor_file (str): Path to receptor file (PDB or PDBQT)
            ligand_file (str): Path to ligand file (SDF or PDBQT)
            receptor_type (str): Either "BACE1" or "PHF6" to set appropriate parameters
        """
        # Set binding pocket parameters based on receptor type
        if receptor_type == "BACE1":
            self.center = (2.5, 6.5, -7.3)  # BACE1 catalytic site
            self.box_size = (20, 20, 20)
        elif receptor_type == "PHF6":
            self.center = (0, 0, 0)  # PHF6 binding region
            self.box_size = (25, 25, 25)
        else:
            raise ValueError("receptor_type must be either 'BACE1' or 'PHF6'")

        # Prepare files if needed
        self.receptor = self._prepare_receptor(receptor_file)
        self.ligand = self._prepare_ligand(ligand_file)
        
        # Initialize simulation parameters
        self.setup_simulation()
    
    def _prepare_receptor(self, receptor_file):
        """Prepare receptor file, converting to PDBQT if necessary."""
        if receptor_file.endswith('.pdb'):
            return prepare_receptor_pdbqt(receptor_file, "data/prepared_receptors")
        return receptor_file
    
    def _prepare_ligand(self, ligand_file):
        """Prepare ligand file, converting to PDBQT if necessary."""
        if ligand_file.endswith('.sdf'):
            return convert_sdf_to_pdbqt(ligand_file, "data/prepared_ligands")
        return ligand_file

    def setup_simulation(self):
        """Set up the simulation environment and parameters."""
        try:
            from vina import Vina

            self.vina = Vina()
            self.vina.set_receptor(self.receptor)
            self.vina.set_ligand_from_file(self.ligand)

            # Configure search space
            self.vina.compute_vina_maps(
                center=list(self.center),
                box_size=list(self.box_size)
            )
            print(f"Successfully set up Vina configuration")
            print(f"Search space center: {self.center}")
            print(f"Search space size: {self.box_size}")
        except ImportError:
            print("Warning: Vina not installed!")

### Single-Thread Implementation


In [5]:
def run_single_thread(self, exhaustiveness=8, n_poses=20):
    """Execute molecular docking using a single thread.

    Args:
        exhaustiveness (int): Search exhaustiveness parameter
        n_poses (int): Number of binding poses to generate

    Returns:
        tuple: (energy_scores, execution_time)
    """
    start_time = time.time()
    
    energy_scores = self.vina.dock(exhaustiveness=exhaustiveness, n_poses=n_poses)

    execution_time = time.time() - start_time
    return energy_scores, execution_time


MolecularDocking.run_single_thread = run_single_thread

### Parallel Implementation


In [6]:
def run_parallel(self, n_threads, exhaustiveness=8, n_poses=20):
    """Execute molecular docking using parallel processing.

    Args:
        n_threads (int): Number of threads to use
        exhaustiveness (int): Search exhaustiveness parameter
        n_poses (int): Number of binding poses to generate

    Returns:
        tuple: (energy_scores, execution_time)
    """
    start_time = time.time()

    # Set OpenMP thread count
    os.environ["OMP_NUM_THREADS"] = str(n_threads)

    def worker_task():
        return self.vina.dock(
            exhaustiveness=exhaustiveness // n_threads, n_poses=n_poses // n_threads
        )

    # Execute parallel docking
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        futures = [executor.submit(worker_task) for _ in range(n_threads)]
        energy_scores = []
        for future in futures:
            energy_scores.extend(future.result())

    execution_time = time.time() - start_time
    return energy_scores, execution_time


MolecularDocking.run_parallel = run_parallel

## Benchmarking Implementation


In [7]:
def benchmark(self, thread_counts=[1, 2, 4, 8], repeats=3):
    """Run performance benchmarks across different thread counts.

    Args:
        thread_counts (list): Thread counts to test
        repeats (int): Number of repetitions per configuration

    Returns:
        dict: Benchmark results including timing and resource usage
    """
    results = {
        "threads": thread_counts,
        "times": [],
        "cpu_usage": [],
        "memory_usage": [],
        "energy_stats": [],
    }

    for n_threads in thread_counts:
        thread_times = []
        thread_cpu = []
        thread_memory = []
        thread_energy = []

        for _ in range(repeats):
            # Monitor system resources
            process = psutil.Process(os.getpid())

            # Run appropriate version
            if n_threads == 1:
                scores, exec_time = self.run_single_thread()
            else:
                scores, exec_time = self.run_parallel(n_threads)

            thread_times.append(exec_time)
            thread_cpu.append(process.cpu_percent())
            thread_memory.append(process.memory_info().rss / 1024 / 1024)  # MB
            thread_energy.append(np.mean(scores))

        # Store average results
        results["times"].append(np.mean(thread_times))
        results["cpu_usage"].append(np.mean(thread_cpu))
        results["memory_usage"].append(np.mean(thread_memory))
        results["energy_stats"].append(np.mean(thread_energy))

    return results


MolecularDocking.benchmark = benchmark

## Visualization


In [8]:
def plot_benchmarks(self, results):
    """Create visualization plots for benchmark results.

    Args:
        results (dict): Benchmark results from benchmark()
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

    # Execution time plot
    ax1.plot(results["threads"], results["times"], marker="o", linewidth=2)
    ax1.set_xlabel("Number of Threads")
    ax1.set_ylabel("Execution Time (s)")
    ax1.set_title("Execution Time Scaling")
    ax1.grid(True)

    # Speedup plot
    speedup = [results["times"][0] / t for t in results["times"]]
    ax2.plot(results["threads"], speedup, marker="o", linewidth=2)
    ax2.plot(results["threads"], results["threads"], "--", label="Linear Speedup")
    ax2.set_xlabel("Number of Threads")
    ax2.set_ylabel("Speedup Factor")
    ax2.set_title("Parallel Speedup")
    ax2.legend()
    ax2.grid(True)

    # Resource usage plots
    ax3.plot(results["threads"], results["cpu_usage"], marker="o", linewidth=2)
    ax3.set_xlabel("Number of Threads")
    ax3.set_ylabel("CPU Usage (%)")
    ax3.set_title("CPU Utilization")
    ax3.grid(True)

    ax4.plot(results["threads"], results["memory_usage"], marker="o", linewidth=2)
    ax4.set_xlabel("Number of Threads")
    ax4.set_ylabel("Memory Usage (MB)")
    ax4.set_title("Memory Usage")
    ax4.grid(True)

    plt.tight_layout()
    plt.show()


MolecularDocking.plot_benchmarks = plot_benchmarks

## Performance Analysis


Edit the arguments in the `MolecularDocking` class constructor to match your input files and desired settings.

In [ ]:
# Initialize docking simulation
docking = MolecularDocking(
    receptor_file="path/to/receptor.pdbqt",
    ligand_file="path/to/ligand.pdbqt",
    center=(0, 0, 0),
    box_size=(20, 20, 20),
)

In [ ]:
# Run benchmarks with multiple thread counts
thread_counts = [1, 2, 4, 8]
results = docking.benchmark(thread_counts=thread_counts, repeats=3)

# Visualize the results
docking.plot_benchmarks(results)

# Calculate and display key performance metrics
speedup = [results["times"][0] / t for t in results["times"]]
efficiency = [s / t for s, t in zip(speedup, results["threads"])]

print("\nPerformance Analysis:")
print("-" * 50)
print(f"{'Threads':<10} {'Time (s)':<12} {'Speedup':<12} {'Efficiency':<12}")
print("-" * 50)
for i, threads in enumerate(results["threads"]):
    print(
        f"{threads:<10} {results['times'][i]:<12.2f} {speedup[i]:<12.2f} {efficiency[i]:<12.2f}"
    )

# Analyze parallel efficiency
plt.figure(figsize=(10, 6))
plt.plot(results["threads"], efficiency, marker="o", linewidth=2)
plt.xlabel("Number of Threads")
plt.ylabel("Parallel Efficiency")
plt.title("Parallel Efficiency vs Thread Count")
plt.grid(True)
plt.show()


def generate_performance_report(results):
    """Generate a detailed analysis of the benchmark results."""
    speedup = [results["times"][0] / t for t in results["times"]]
    max_speedup = max(speedup)
    optimal_threads = results["threads"][speedup.index(max_speedup)]
    avg_cpu_usage = np.mean(results["cpu_usage"])

    print("\nDetailed Performance Report:")
    print("-" * 50)
    print(f"Optimal Thread Count: {optimal_threads}")
    print(f"Maximum Speedup: {max_speedup:.2f}x")
    print(f"Average CPU Usage: {avg_cpu_usage:.1f}%")
    print(
        f"Memory Usage Range: {min(results['memory_usage']):.1f} - {max(results['memory_usage']):.1f} MB"
    )

    # Analyze scaling efficiency
    if max_speedup < optimal_threads * 0.8:
        print("\nNote: Sublinear scaling observed - consider investigating:")
        print("- Memory bandwidth limitations")
        print("- Load balancing issues")
        print("- Thread synchronization overhead")

In [ ]:
def generate_performance_report(results):
    """Generate a detailed analysis of the benchmark results."""
    speedup = [results["times"][0] / t for t in results["times"]]
    max_speedup = max(speedup)
    optimal_threads = results["threads"][speedup.index(max_speedup)]
    avg_cpu_usage = np.mean(results["cpu_usage"])

    print("\nDetailed Performance Report:")
    print("-" * 50)
    print(f"Optimal Thread Count: {optimal_threads}")
    print(f"Maximum Speedup: {max_speedup:.2f}x")
    print(f"Average CPU Usage: {avg_cpu_usage:.1f}%")
    print(
        f"Memory Usage Range: {min(results['memory_usage']):.1f} - {max(results['memory_usage']):.1f} MB"
    )

    # Analyze scaling efficiency
    if max_speedup < optimal_threads * 0.8:
        print("\nNote: Sublinear scaling observed - consider investigating:")
        print("- Memory bandwidth limitations")
        print("- Load balancing issues")
        print("- Thread synchronization overhead")

generate_performance_report(results)